# Read Dataset

In [1]:
import pandas as pd

data = pd.read_excel("data/diy/sample_data_1.xlsx")
data.head()

,conversation_id_str,created_at,favorite_count,full_text,id_str,image_url,in_reply_to_screen_name,lang,location,quote_count,reply_count,retweet_count,tweet_url,user_id_str,username
0,1924499205722084096,Mon May 19 16:15:40 +0000 2025,0,Palang Pintu Pelintasan Terbuka Saat KA Maliob...,1924499205722084096,https://pbs.twimg.com/media/GrUx1A-XIAAwSH_.jpg,NaN,in,NaN,0,0,0,https://x.com/undefined/status/192449920572208...,118371948,NaN
1,1924455275739693056,Mon May 19 13:21:07 +0000 2025,6,ASN Pemkab Magetan Jadi Korban Meninggal Kecel...,1924455275739693056,https://pbs.twimg.com/media/GrUJ3-rWoAA0y4T.jpg,NaN,in,NaN,1,0,0,https://x.com/undefined/status/192445527573969...,118371948,NaN
2,1924350383109165056,Mon May 19 12:19:00 +0000 2025,0,@regar_op0sisi Harusnya di panggilin damkar in...,1924439646890820096,NaN,regar_op0sisi,in,NaN,0,0,0,https://x.com/undefined/status/192443964689082...,280381391,NaN
3,1924388809271034112,Mon May 19 08:57:00 +0000 2025,1,Kronologi Kecelakaan Kereta di Magetan Palang ...,1924388809271034112,https://pbs.twimg.com/media/GrTNbIsW0AA_PCC.jpg,NaN,in,NaN,0,0,0,https://x.com/undefined/status/192438880927103...,118371948,NaN
4,1924384204156026880,Mon May 19 08:38:42 +0000 2025,1,Kecelakaan KA Malioboro Ekspres di Magetan Tew...,1924384204156026880,https://pbs.twimg.com/media/GrTJPAyW0AAZg7J.jpg,NaN,in,NaN,0,0,0,https://x.com/undefined/status/192438420415602...,118371948,NaN


In [2]:
# Contoh: styling kolom agar wrap
def display_fullscreen_wrap(df):
    return df.style.set_properties(
        **{
            "white-space": "pre-wrap",  # wrap isi cell
            "word-break": "break-word",  # pecah di mana saja kalau kepanjangan
            "width": "900px",  # bisa diganti sesuai kebutuhan
            "max-width": "700px",  # atur lebar kolom maksimal
        }
    )

# EDA

In [3]:
data.columns

Index(['conversation_id_str', 'created_at', 'favorite_count', 'full_text',
       'id_str', 'image_url', 'in_reply_to_screen_name', 'lang', 'location',
       'quote_count', 'reply_count', 'retweet_count', 'tweet_url',
       'user_id_str', 'username'],
      dtype='object')

### Summary

In [4]:
info = data.info()
print("Summary :", info)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7744 entries, 0 to 7743
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   conversation_id_str      7744 non-null   int64  
 1   created_at               7744 non-null   object 
 2   favorite_count           7744 non-null   int64  
 3   full_text                7744 non-null   object 
 4   id_str                   7744 non-null   int64  
 5   image_url                1567 non-null   object 
 6   in_reply_to_screen_name  3735 non-null   object 
 7   lang                     7744 non-null   object 
 8   location                 0 non-null      float64
 9   quote_count              7744 non-null   int64  
 10  reply_count              7744 non-null   int64  
 11  retweet_count            7744 non-null   int64  
 12  tweet_url                7744 non-null   object 
 13  user_id_str              7744 non-null   int64  
 14  username                

### Find Missing Values

In [5]:
# menghitung dan menampilkan missing values
print("Jumlah missing values disetiap kolom:\n", data.isnull().sum())

Jumlah missing values disetiap kolom:
 conversation_id_str           0
created_at                    0
favorite_count                0
full_text                     0
id_str                        0
image_url                  6177
in_reply_to_screen_name    4009
lang                          0
location                   7744
quote_count                   0
reply_count                   0
retweet_count                 0
tweet_url                     0
user_id_str                   0
username                   7744
dtype: int64


### Delete unnecessary columns and does not provide any information

In [6]:
data = data[["full_text"]]
data.head()

,full_text
0,Palang Pintu Pelintasan Terbuka Saat KA Maliob...
1,ASN Pemkab Magetan Jadi Korban Meninggal Kecel...
2,@regar_op0sisi Harusnya di panggilin damkar in...
3,Kronologi Kecelakaan Kereta di Magetan Palang ...
4,Kecelakaan KA Malioboro Ekspres di Magetan Tew...


### Check Duplicate Data that contain in the content column

In [7]:
data.duplicated().sum()

np.int64(838)

# Data Preparation

### Case Folding

In [8]:
# merubah jenis huruf menjadi huruf kecil
data["full_text"] = data["full_text"].str.lower()
data.head()

,full_text
0,palang pintu pelintasan terbuka saat ka maliob...
1,asn pemkab magetan jadi korban meninggal kecel...
2,@regar_op0sisi harusnya di panggilin damkar in...
3,kronologi kecelakaan kereta di magetan palang ...
4,kecelakaan ka malioboro ekspres di magetan tew...


## Cleaning Data

### Emoji To Word

In [9]:
import pandas as pd
from indoNLP.preprocessing import (
    pipeline,
    replace_word_elongation,
    replace_slang,
    remove_html,
    remove_url,
    emoji_to_words
)
# Apply emoji_to_words to the text column
data["full_text"] = data["full_text"].apply(lambda x: emoji_to_words(str(x), lang="id"))
display_fullscreen_wrap(data.head())

,full_text
0,palang pintu pelintasan terbuka saat ka malioboro ekspres melintas kecelakaan pun tak bisa dihindari https://t.co/2rau13j0cp https://t.co/ouql7ahsug
1,asn pemkab magetan jadi korban meninggal kecelakaan ka malioboro ekspres https://t.co/hin963viyt https://t.co/b7bvzvythg
2,@regar_op0sisi harusnya di panggilin damkar ini.. gercep langsung ke lokasi mengamankan kecelakaan
3,kronologi kecelakaan kereta di magetan palang pintu diduga terbuka saat ka malioboro ekspres melintas https://t.co/f3sgxfpbfv https://t.co/7cog3mw3q2
4,kecelakaan ka malioboro ekspres di magetan tewaskan 4 orang ini kronologinya https://t.co/azsah90363 https://t.co/cgjchkd0tu


### Remove HTML

In [10]:
# data["full_text"] = data["full_text"].apply(lambda x: remove_html(str(x)))
# display_fullscreen_wrap(data.head())

### Remove URL

In [11]:
# data["full_text"] = data["full_text"].apply(lambda x: remove_url(str(x)))
# display_fullscreen_wrap(data.head())

### Replace Slang

In [12]:
# data["full_text"] = data["full_text"].apply(lambda x: replace_slang(str(x)))
# display_fullscreen_wrap(data.head())

### Replace Word Elongation

In [13]:
data["full_text"] = data["full_text"].apply(lambda x: replace_word_elongation(str(x)))
display_fullscreen_wrap(data.head())

,full_text
0,palang pintu pelintasan terbuka saat ka malioboro ekspres melintas kecelakaan pun tak bisa dihindari https://t.co/2rau13j0cp https://t.co/ouql7ahsug
1,asn pemkab magetan jadi korban meninggal kecelakaan ka malioboro ekspres https://t.co/hin963viyt https://t.co/b7bvzvythg
2,@regar_op0sisi harusnya di panggilin damkar ini.. gercep langsung ke lokasi mengamankan kecelakaan
3,kronologi kecelakaan kereta di magetan palang pintu diduga terbuka saat ka malioboro ekspres melintas https://t.co/f3sgxfpbfv https://t.co/7cog3mw3q2
4,kecelakaan ka malioboro ekspres di magetan tewaskan 4 orang ini kronologinya https://t.co/azsah90363 https://t.co/cgjchkd0tu


### Replace User Mentions

In [14]:
import re

# Function to replace URLs and user mentions
def replace_urls_and_mentions(text):
    # Replace URLs with HTTPURL
    text = re.sub(
        r"http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+",
        "HTTPURL",
        text,
    )
    text = re.sub(
        r"www\.(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+",
        "HTTPURL",
        text,
    )

    # Replace user mentions with @USER
    text = re.sub(r"@[A-Za-z0-9_]+", "@USER", text)

    return text

# Apply URL and mention replacement
data["full_text"] = data["full_text"].apply(lambda x: replace_urls_and_mentions(str(x)))
display_fullscreen_wrap(data.head())

,full_text
0,palang pintu pelintasan terbuka saat ka malioboro ekspres melintas kecelakaan pun tak bisa dihindari HTTPURL HTTPURL
1,asn pemkab magetan jadi korban meninggal kecelakaan ka malioboro ekspres HTTPURL HTTPURL
2,@USER harusnya di panggilin damkar ini.. gercep langsung ke lokasi mengamankan kecelakaan
3,kronologi kecelakaan kereta di magetan palang pintu diduga terbuka saat ka malioboro ekspres melintas HTTPURL HTTPURL
4,kecelakaan ka malioboro ekspres di magetan tewaskan 4 orang ini kronologinya HTTPURL HTTPURL


In [15]:
# Check Ollama status and fix connection issues

import requests
import subprocess
import json

def check_ollama_status():
    """Check if Ollama is running and what models are available"""
    try:
        # Check if Ollama is running
        response = requests.get("http://localhost:11434/api/tags", timeout=5)
        if response.status_code == 200:
            models = response.json()
            print("✅ Ollama is running!")
            print("Available models:")
            for model in models.get("models", []):
                print(f"  - {model['name']}")
            return True, models.get("models", [])
        else:
            print(f"❌ Ollama API returned status: {response.status_code}")
            return False, []
    except requests.exceptions.ConnectionError:
        print("❌ Cannot connect to Ollama. Is it running?")
        print("To start Ollama, run: ollama serve")
        return False, []
    except Exception as e:
        print(f"❌ Error checking Ollama: {e}")
        return False, []


def start_ollama():
    """Try to start Ollama if it's not running"""
    try:
        print("🔄 Trying to start Ollama...")
        subprocess.Popen(
            ["ollama", "serve"], stdout=subprocess.PIPE, stderr=subprocess.PIPE
        )
        print("✅ Ollama started! Wait a few seconds for it to initialize...")
        import time

        time.sleep(5)
        return True
    except Exception as e:
        print(f"❌ Failed to start Ollama: {e}")
        print("Please start Ollama manually by running 'ollama serve' in terminal")
        return False

# Check Ollama status
print("Checking Ollama status...")
is_running, available_models = check_ollama_status()

if not is_running:
    print("\n🔧 Ollama is not running. Trying to start it...")
    if start_ollama():
        # Recheck after starting
        import time

        time.sleep(3)
        is_running, available_models = check_ollama_status()

if is_running:
    if available_models:
        print("\n🎉 Ollama is running and available models are:")
        model = "qwen2.5:14b"  # Default model
        # Update the function with correct model
        def classify_hate_speech_with_ollama(text, model=model):
            """
            Classify text for hate speech using Ollama Llama3
            Returns: dict with 'is_hate_speech', 'target_type', 'confidence', 'reasoning'
            """

            prompt = f"""
Analisis teks berikut untuk menentukan apakah mengandung hate speech (ucapan kebencian) atau tidak.

Hate Speech adalah tindakan komunikasi berupa provokasi, hasutan, atau hinaan kepada individu/kelompok berdasarkan ras, warna kulit, etnis, gender, cacat, orientasi seksual, kewarganegaraan, agama, status sosial ekonomi, pandangan politik, atau karakteristik identitas lainnya.

Kriteria Hate Speech:
1. Mengandung kata-kata kasar, cercaan, atau makian yang ditujukan pada kelompok tertentu
2. Menyebarkan kebencian atau diskriminasi terhadap identitas seseorang
3. Menghasut atau memprovokasi tindakan kekerasan terhadap individu/kelompok
4. Merendahkan atau menghina berdasarkan karakteristik pribadi atau identitas
5. Mengandung ancaman atau intimidasi terhadap kelompok tertentu

Teks: "{text}"

Berikan analisis dalam format JSON:
{{
    "is_hate_speech": true/false,
    "target_type": "individual/group/none",
    "confidence": "high/medium/low",
    "reasoning": "penjelasan singkat mengapa diklasifikasikan demikian"
}}

Jawab hanya dengan JSON, tanpa penjelasan tambahan.
"""

            try:
                response = requests.post(
                    "http://localhost:11434/api/generate",
                    json={
                        "model": model,
                        "prompt": prompt,
                        "stream": False,
                        "options": {
                            "temperature": 0.1,
                            "top_p": 0.9,
                        },
                    },
                    timeout=60,
                )

                if response.status_code == 200:
                    result = response.json()
                    generated_text = result["response"].strip()

                    try:
                        import re

                        json_match = re.search(r"\{.*\}", generated_text, re.DOTALL)
                        if json_match:
                            json_str = json_match.group()
                            classification = json.loads(json_str)
                            return classification
                        else:
                            return {
                                "is_hate_speech": False,
                                "target_type": "none",
                                "confidence": "low",
                                "reasoning": "Failed to parse response",
                            }
                    except json.JSONDecodeError:
                        return {
                            "is_hate_speech": False,
                            "target_type": "none",
                            "confidence": "low",
                            "reasoning": "Invalid JSON response",
                        }
                else:
                    return {
                        "is_hate_speech": False,
                        "target_type": "none",
                        "confidence": "low",
                        "reasoning": f"API error: {response.status_code}",
                    }

            except Exception as e:
                return {
                    "is_hate_speech": False,
                    "target_type": "none",
                    "confidence": "low",
                    "reasoning": f"Error: {str(e)}",
                }

        # Test the function with a sample text
        sample_text = "Selamat pagi, bagaimana kabar Anda hari ini?"
        test_result = classify_hate_speech_with_ollama(sample_text)
        print("\n🧪 Test classification result:")
        print(json.dumps(test_result, indent=2, ensure_ascii=False))

    else:
        print("❌ Could not setup any model")
else:
    print("\n❌ Please start Ollama manually:")
    print("1. Open terminal")
    print("2. Run: ollama serve")
    print("3. In another terminal, run: ollama pull llama3.1:8b")
    print("4. Then run this cell again")

Checking Ollama status...
✅ Ollama is running!
Available models:
  - qwen3:14b
  - qwen2.5:14b
  - llama3.1:8b
  - deepseek-r1:14b

🎉 Ollama is running and available models are:

🧪 Test classification result:
{
  "is_hate_speech": false,
  "target_type": "none",
  "confidence": "high",
  "reasoning": "Teks tersebut merupakan salam yang sopan dan netral, tidak mengandung unsur provokasi, hinaan, atau diskriminasi terhadap individu atau kelompok tertentu."
}


In [ ]:
# Fixed Multi-Category Hate Speech Classification
import json


def classify_hate_speech_detailed(
    text, model="qwen2.5:14b"
):  # ✅ Gunakan model yang tersedia
    """
    Classify text for hate speech with detailed categories
    Returns: dict with multiple binary labels for different types of hate speech
    """

    prompt = f"""
Kamu adalah ahli analisis hate speech bahasa Indonesia. Analisis teks berikut dengan SANGAT HATI-HATI.

DEFINISI HATE SPEECH:
- Ucapan yang mengandung KEBENCIAN, DISKRIMINASI, atau PENGHINAAN terhadap individu/kelompok
- Berdasarkan ras, agama, gender, etnis, orientasi seksual, disabilitas, dll
- Menghasut kekerasan atau menciptakan permusuhan

YANG BUKAN HATE SPEECH:
- Berita kecelakaan, bencana, kriminal
- Kritik konstruktif
- Keluhan layanan publik
- Diskusi politik normal

Teks: "{text}"

Analisis dengan format JSON (nilai 1=ada, 0=tidak ada):
{{
    "HS": 0,
    "HS_INDIVIDU": 0,
    "HS_KELOMPOK": 0,
    "HS_RAS": 0,
    "HS_GENDER": 0,
    "HS_AGAMA": 0,
    "HS_POLITIK": 0,
    "HS_FISIK": 0,
    "HS_SOSIAL": 0,
    "confidence": "high",
    "reasoning": "penjelasan detail"
}}

HANYA jawab JSON, tidak ada teks lain.
"""

    try:
        response = requests.post(
            "http://localhost:11434/api/generate",
            json={
                "model": model,
                "prompt": prompt,
                "stream": False,
                "options": {
                    "temperature": 0.3,  # ✅ Naikkan sedikit untuk fleksibilitas
                    "top_p": 0.9,
                },
            },
            timeout=60,
        )

        if response.status_code == 200:
            result = response.json()
            generated_text = result["response"].strip()

            try:
                import re

                json_match = re.search(r"\{.*\}", generated_text, re.DOTALL)
                if json_match:
                    json_str = json_match.group()
                    classification = json.loads(json_str)

                    # Ensure all required fields exist with default values
                    default_result = {
                        "HS": 0,
                        "HS_INDIVIDU": 0,
                        "HS_KELOMPOK": 0,
                        "HS_RAS": 0,
                        "HS_GENDER": 0,
                        "HS_AGAMA": 0,
                        "HS_POLITIK": 0,
                        "HS_FISIK": 0,
                        "HS_SOSIAL": 0,
                        "confidence": "low",
                        "reasoning": "No classification",
                    }

                    # Update with actual results
                    default_result.update(classification)
                    return default_result
                else:
                    return default_result
            except json.JSONDecodeError:
                return default_result
        else:
            return default_result

    except Exception as e:
        return {
            "HS": 0,
            "HS_INDIVIDU": 0,
            "HS_KELOMPOK": 0,
            "HS_RAS": 0,
            "HS_GENDER": 0,
            "HS_AGAMA": 0,
            "HS_POLITIK": 0,
            "HS_FISIK": 0,
            "HS_SOSIAL": 0,
            "confidence": "low",
            "reasoning": f"Error: {str(e)}",
        }


# Test dengan teks yang JELAS BUKAN hate speech (FIXED - no double output)
test_texts = [
    "palang pintu pelintasan terbuka saat ka malioboro ekspres melintas kecelakaan pun tak bisa dihindari",
    "selamat pagi semua, semoga hari ini menyenangkan",
    "dasar orang hitam jelek kayak monyet",  # ini baru hate speech
]

print("Testing full JSON output:")
for i, text in enumerate(test_texts):
    result = classify_hate_speech_detailed(text)
    print(f"\n=== Text {i+1} ===")
    print(f"Text: {text[:50]}...")
    print("Full JSON Result:")
    print(json.dumps(result, indent=2, ensure_ascii=False))
    print("-" * 70)

Testing full JSON output:

=== Text 1 ===
Text: palang pintu pelintasan terbuka saat ka malioboro ...
Full JSON Result:
{
  "HS": 0,
  "HS_INDIVIDU": 0,
  "HS_KELOMPOK": 0,
  "HS_RAS": 0,
  "HS_GENDER": 0,
  "HS_AGAMA": 0,
  "HS_POLITIK": 0,
  "HS_FISIK": 0,
  "HS_SOSIAL": 0,
  "confidence": "high",
  "reasoning": "Teks tersebut hanya menggambarkan kejadian kecelakaan yang terjadi akibat kondisi pelintasan dan kereta api, tanpa unsur hate speech."
}
----------------------------------------------------------------------


In [ ]:
# Test klasifikasi dengan 3 data pertama
import time
import pandas as pd

print("=== Testing dengan 3 data pertama ===")
test_data = data.head(3).copy()

# Tambahkan kolom-kolom hasil klasifikasi
classification_columns = [
    "HS",
    "HS_INDIVIDU",
    "HS_KELOMPOK",
    "HS_RAS",
    "HS_GENDER",
    "HS_AGAMA",
    "HS_POLITIK",
    "HS_FISIK",
    "HS_SOSIAL",
    "confidence",
    "reasoning",
]

for col in classification_columns:
    test_data[col] = None

# Klasifikasi 3 data pertama
for idx, row in test_data.iterrows():
    print(f"\n🔄 Memproses data ke-{idx+1}...")
    text = row["full_text"]
    print(f"Text: {text[:100]}...")

    # Klasifikasi
    result = classify_hate_speech_detailed(text)

    # Simpan hasil ke dataframe
    for col in classification_columns:
        test_data.at[idx, col] = result.get(
            col, 0 if col.startswith("HS") else "unknown"
        )

    print(f"HS: {result['HS']}, Confidence: {result['confidence']}")

    # Delay untuk menghindari overload
    time.sleep(2)

print("\n=== Hasil Testing 3 Data ===")
display_fullscreen_wrap(
    test_data[
        ["full_text", "HS", "HS_INDIVIDU", "HS_KELOMPOK", "confidence", "reasoning"]
    ]
)

In [ ]:
# Setelah testing berhasil, proses semua data
print("=== Memproses SEMUA data ===")
print(f"Total data yang akan diproses: {len(data)}")

# Buat copy data dengan kolom klasifikasi
final_data = data.copy()

# Tambahkan kolom hasil klasifikasi
classification_columns = [
    "HS",
    "HS_INDIVIDU",
    "HS_KELOMPOK",
    "HS_RAS",
    "HS_GENDER",
    "HS_AGAMA",
    "HS_POLITIK",
    "HS_FISIK",
    "HS_SOSIAL",
    "confidence",
    "reasoning",
]

for col in classification_columns:
    final_data[col] = None

# Progress tracking
total_data = len(final_data)
processed = 0
start_time = time.time()

print("🚀 Mulai klasifikasi semua data...")

for idx, row in final_data.iterrows():
    processed += 1
    text = row["full_text"]

    # Show progress
    if processed % 10 == 0 or processed <= 5:
        elapsed = time.time() - start_time
        avg_time = elapsed / processed
        remaining = (total_data - processed) * avg_time
        print(
            f"Progress: {processed}/{total_data} ({processed/total_data*100:.1f}%) - ETA: {remaining/60:.1f} menit"
        )
        print(f"Current text: {text[:80]}...")

    # Klasifikasi
    result = classify_hate_speech_detailed(text)

    # Simpan hasil
    for col in classification_columns:
        final_data.at[idx, col] = result.get(
            col, 0 if col.startswith("HS") else "unknown"
        )

    # Delay untuk stability
    time.sleep(1)

print(f"\n✅ Selesai! Total waktu: {(time.time() - start_time)/60:.1f} menit")

In [ ]:
# Simpan hasil klasifikasi
import os

# Buat folder output jika belum ada
output_dir = "data/diy"
os.makedirs(output_dir, exist_ok=True)

# Simpan ke Excel
output_file = f"{output_dir}/hate_speech_classification_results.xlsx"
final_data.to_excel(output_file, index=False)
print(f"✅ Data berhasil disimpan ke: {output_file}")

# Simpan juga ke CSV untuk backup
csv_file = f"{output_dir}/hate_speech_classification_results.csv"
final_data.to_csv(csv_file, index=False)
print(f"✅ Backup CSV disimpan ke: {csv_file}")

# Show summary statistics
print("\n=== SUMMARY HASIL KLASIFIKASI ===")
print(f"Total data: {len(final_data)}")
print(
    f"Hate Speech terdeteksi: {final_data['HS'].sum()} ({final_data['HS'].sum()/len(final_data)*100:.1f}%)"
)
print(
    f"Non Hate Speech: {len(final_data) - final_data['HS'].sum()} ({(len(final_data) - final_data['HS'].sum())/len(final_data)*100:.1f}%)"
)

print("\n=== Breakdown per kategori ===")
for col in [
    "HS_INDIVIDU",
    "HS_KELOMPOK",
    "HS_RAS",
    "HS_GENDER",
    "HS_AGAMA",
    "HS_POLITIK",
    "HS_FISIK",
    "HS_SOSIAL",
]:
    count = final_data[col].sum()
    print(f"{col}: {count} ({count/len(final_data)*100:.1f}%)")

print("\n=== Confidence level ===")
print(final_data["confidence"].value_counts())

# Show sample results
print("\n=== Sample Hasil Klasifikasi ===")
display_fullscreen_wrap(
    final_data[
        ["full_text", "HS", "HS_INDIVIDU", "HS_KELOMPOK", "confidence", "reasoning"]
    ].head(10)
)